In [1]:
from os.path import join
import numpy as np
import pandas as pd
from numpy import linalg as LA
from torch.autograd import Variable
import torchvision
import sys
import torch
import os
import scipy.io
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

## GPU or CPU
GPU = True
if GPU:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    print(f"GPU(s): {torch.cuda.device_count()}x {torch.cuda.get_device_name()}")
    dtype = torch.cuda.FloatTensor
else:
    dtype = torch.FloatTensor
    print("CPU")
eps = torch.finfo(torch.float32).eps
torch.cuda.empty_cache()

GPU(s): 1x NVIDIA GeForce RTX 3060


In [2]:
def np_to_tensor(Mat):
    return torch.from_numpy(Mat)

def np_to_var(Mat, dtype = torch.cuda.FloatTensor):
    return Variable(np_to_tensor(Mat)[None, :])


# Read Dataset

def readDataset(selected):

    edgelist = np.genfromtxt(f'Dataset/Celegansneural.edges').astype(int)
    e = len(edgelist)
    n = np.max(edgelist)
    A = torch.zeros([n, n])
    Atr = torch.zeros([n, n])

    # Percent of delete edges
    per = 0.9
    oe = np.round(per * e).astype(int)

    train = np.random.choice(e, oe, replace=False)

    for i in range(e):
        A[edgelist[i, 0] - 1, edgelist[i, 1] - 1] = 1
        A[edgelist[i, 1] - 1, edgelist[i, 0] - 1] = 1

    for i in train:
        Atr[edgelist[i, 0] - 1, edgelist[i, 1] - 1] = 1
        Atr[edgelist[i, 1] - 1, edgelist[i, 0] - 1] = 1

    Ate = A - Atr
    dataset_name = 'Celegansneural'
    print("Dataset_name: ",dataset_name,"\n#node => ",n, "#edge => ",e,'\n')
    return A, Atr, Ate, dataset_name, n, e

In [3]:
# precision Mesure
def precision(V, Ate,Apred):
    
    n = Ate.shape[0]
    Ap = (1-V) * Apred
    L1 = torch.sum(Ate).int()
    
    ap = torch.reshape(Ap, (n**2,1))
    ate = torch.reshape(Ate, (n**2,1))
    values, indices = ap.topk(k=L1,dim=0) 
    Ind = indices[values!=0]
    app = torch.zeros(n**2,1)
    app[Ind] = 1
    
    l1 = torch.sum(app.cuda() * ate.cuda())
    prec = l1/L1
    return prec.cpu()

In [8]:
# A Regularized Convex Nonnegative Matrix Factorization Model for signed network analysis (RC-NMF)

A, Atr, Ate, dataset_name, n, e = readDataset(1)
[F,N] = A.shape
A = A.type(dtype)
Ate = Ate.type(dtype)
V = Atr.type(dtype)
# print ('Size of ATrain: ' ,V.shape)
degree = torch.sum(V , 0)

itere = 200
lambd = 3
gamma = 8
K = 30

W = abs(np.random.normal(0,1, size = (F,K)))
W = np_to_var(W)[0].type(dtype)

H = abs(np.random.normal(0,1,size = (N,K)))
H = np_to_var(H)[0].type(dtype)

epsW = torch.ones([N,K]) * torch.finfo(torch.float32).eps
epsH = torch.ones([N,K]) * torch.finfo(torch.float32).eps

ATA  = V.T @ V
ATAp = (torch.abs(ATA) + ATA) / 2
ATAn = (torch.abs(ATA) - ATA) / 2
NN   = torch.ones([K,K]).type(dtype)
D    = torch.diag(torch.sum(V,0))

for i in range(1,itere):
    Psi1 = ATAp @ W
    Psi2 = ATAn @ W

    Wup = ATAp @ H + Psi2 @ H.T @ H
    Wdo = ATAn @ H + Psi1 @ H.T @ H
    W = W * (Wup / ((Wdo + torch.finfo(torch.float32).eps)))**(1/2)

    Psi1 = ATAp @ W
    Psi2 = ATAn @ W
    Hup = Psi1 + H @ W.T @ Psi2 + lambd * (V @ H)
    Hdo = Psi2 + H @ W.T @ Psi1 + lambd * (D @ H) + gamma * (H @ NN)
    H = H * ((Hup / (Hdo + torch.finfo(torch.float32).eps)))**(1/2)

Apred = V @ W @ H.T
prec = precision(V, Ate, Apred)
print('RC-NMF Precision= ',prec)

Dataset_name:  Celegansneural 
#node =>  297 #edge =>  2345 

RC-NMF Precision=  tensor(0.1553)
